In [5]:
from fontMatching.fontGen import *
from fontMatching.model import *
from visualize import optimize2
from os import walk
import json
from ocr import interogate, add_rect 
import math
from bs4 import BeautifulSoup
from svgpathtools import svg2paths2
import webcolors
from fontMatching.scoring import *


# Bounds are formatted (left, right, top, bottom) !!!

In [6]:

def formatted_get_text(image_path, texts):
    feature = get_text_feature(image_path, texts)

    for k,v in feature.items():
        try:
            color_hex = webcolors.hex_to_rgb(v["color"])
            background_hex = webcolors.hex_to_rgb(v["backgroundColor"])
            v["color"] = (color_hex.red, color_hex.green, color_hex.blue)
            v["backgroundColor"] = (background_hex.red, background_hex.green, background_hex.blue)
        except:
            print(feature)
            v["color"] = (0, 0, 0)
            v["backgroundColor"] = (255, 255, 255)
    return feature


In [7]:
text = {
    "text": "NEW",
    "x": 4,
    "y": 0,
    "color": (0,255,0),
    "fontSize": "18",
    "backgroundColor": (0,0,0),
}
images = create_text(text, (200, 20))
for i in images:
    display(i)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\CaveatBrush-Regular.ttf',
 25)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Lexend-Black.ttf',
 28)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Lexend-Bold.ttf',
 28)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Lexend-ExtraBold.ttf',
 28)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Lexend-ExtraLight.ttf',
 28)

...

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\LibreBaskerville-Italic.ttf',
 25)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\LibreBaskerville-Regular.ttf',
 25)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Pacifico-Regular.ttf',
 24)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Tangerine-Bold.ttf',
 29)

(<PIL.Image.Image image mode=RGBA size=200x20>,
 'c:\\project\\svg-typography-rust\\fontMatching\\fonts\\Tangerine-Regular.ttf',
 31)

In [ ]:
for _,_,files in walk("./svgs"):
    for file in files:
        image_path = f"./pngs/{file}.png"
        id_svg_path = f"./id_svg/{file}_withID.svg"

        json_dict = add_rect(file)
        ocr_dict = interogate(image_path)

        if (not ocr_dict): continue

        with open(id_svg_path) as f:
            soup = BeautifulSoup(f, "xml")
            root = soup.find("svg")
            _,_, attr = svg2paths2(id_svg_path)

        dim = attr["viewBox"].split(" ")

        svg_width = float(dim[2].replace("px",""))
        svg_height = float(dim[3].replace("px",""))

        img_width, img_height = int(attr["width"].replace("px", "")), int(attr["height"].replace("px", ""))

        m,n = len(json_dict), len(ocr_dict)

        json_map = {i : key for key, i in zip(list(json_dict.keys()), range(m))}
        ocr_map = {i : key for key, i in zip(list(ocr_dict.keys()), range(n))}

        json_map_inv = {v: k for k, v in json_map.items()}
        ocr_map_inv = {v: k for k, v in ocr_map.items()}

        bounds_json = list(json_dict.values())
        bounds_ocr = list(ocr_dict.values())

        bounds_ocr_svg = [(left * svg_width, right * svg_width, top * svg_height, bottom * svg_height) for left,right,top,bottom in bounds_ocr]
        bounds_ocr_png = [(left * img_width, right * img_width, top * img_height, bottom * img_height) for left,right,top,bottom in bounds_ocr]

        center_json = [((b[1] + b[0]) / 2, (b[2] + b[3]) / 2) for b in bounds_json]
        center_ocr = [((b[1] + b[0]) / 2, (b[2] +  b[3]) / 2) for b in bounds_ocr_svg]

        d = [[ 0 for i in range(n) ] for j in range(m)]


        for i in range(m):
            x1,y1 = center_json[i][0], center_json[i][1]
            for j in range(n):
                x2,y2 = center_ocr[j][0], center_ocr[j][1]

                d[i][j] = math.sqrt((x2-x1) ** 2 + (y2-y1)**2)

        x = [ [0] * (n) for i in range(m) ]
        
        optimize2(bounds_ocr_svg, center_json, x)

        group = [[] for _ in range(n)]
        for i in range(m):
            if max(x[i]):
                group[x[i].index(1)].append(json_map[i])

        # current_feature = formatted_get_text(image_path, list(ocr_dict.keys()))

        new_texts = {}
        fonts_import = []

        for i in range(0, n):
            current_text = ocr_map[i]
            current_bound = bounds_ocr_png[i]

            cropped = crop_image(image_path, current_bound)

            bg_color, text_color = get_color(cropped)

            text = {
                "text": current_text,
                "color": webcolors.rgb_to_hex((text_color[0], text_color[1], text_color[2])),
                "backgroundColor": webcolors.rgb_to_hex((bg_color[0], bg_color[1], bg_color[2])),
            }

            images = create_text(text, (round(current_bound[1] - current_bound[0]), round(current_bound[3] - current_bound[2])))
            matches = hog_scoring(cropped, images)

            best = matches[0]
            font_base64 = f"data:font/ttf;base64,{font_to_base64(best[1])}"
            font_name = best[1].split("\\")[-1].split(".")[0]
            print(font_name)
            text.update({
                "fontSize": best[2] / img_height * svg_height,
                "fontFamily": font_name,
            })
            new_texts[current_text] = text
            fonts_import.append((font_base64, font_name))

        style_tag = soup.find("style")

        if not style_tag:
            style_tag = soup.new_tag("style")
            root.insert(0, style_tag)
        
        for font_base64, font_name in fonts_import:
            style_tag.insert(0, f"@font-face {{font-family: {font_name}; src: url('{font_base64}') format('truetype');}}")

        for text_index, e_list in enumerate(group):
            for element_id in e_list:
                e = soup.find(id=element_id)
                if e: e.decompose()
            
            text_content = ocr_map[text_index]
            text_element = new_texts[text_content]

            print(text_element)

            x,y = center_ocr[text_index]

            replacement = soup.new_tag("text",x=x,y=y, style=f"font-family: {text_element['fontFamily']}; font-size: {text_element['fontSize']}px; fill: {text_element['color']};")
            replacement["text-anchor"] = "middle"
            replacement['dominant-baseline'] = 'central'
            replacement.insert(0, text_content)
   
   
            root.append(replacement)
        
        with open(f"./finalized_svg/{file}_final.svg", "wb") as wf:
            wf.write(soup.prettify("utf-8"))

Lexend-ExtraLight
LibreBaskerville-Bold
Pacifico-Regular
Tangerine-Bold
{'text': 'NEW', 'color': '#1b1a19', 'backgroundColor': '#f9f2ee', 'fontSize': 104.99999909259259, 'fontFamily': 'Lexend-ExtraLight'}
{'text': 'POST', 'color': '#191714', 'backgroundColor': '#f9f2ee', 'fontSize': 92.24999920277777, 'fontFamily': 'LibreBaskerville-Bold'}
{'text': 'swipe', 'color': '#a87e45', 'backgroundColor': '#f9f2ee', 'fontSize': 76.49999933888888, 'fontFamily': 'Pacifico-Regular'}
{'text': 'up', 'color': '#9d743e', 'backgroundColor': '#f9f2ee', 'fontSize': 159.74999861944445, 'fontFamily': 'Tangerine-Bold'}
Lexend-Bold
Lexend-SemiBold
Lexend-SemiBold
Lexend-Bold
Lexend-SemiBold
Lexend-ExtraBold
LibreBaskerville-Regular
Lexend-Light
Lexend-Light
Lexend-Light
LibreBaskerville-Regular
LibreBaskerville-Regular
Lexend-ExtraLight
Lexend-ExtraLight
Lexend-ExtraLight
Lexend-Thin
LibreBaskerville-Regular
Lexend-ExtraLight
Lexend-Light
CaveatBrush-Regular
LibreBaskerville-Regular
Lexend-Medium
Lexend-SemiB

: 